In [1]:
from math import log10 as log
from re import split
from glob import glob
import scipy.optimize as spopt
from scipy.optimize import fmin
import matplotlib.pyplot as plt
import numpy as np
import os
from os.path import join as opj
from natsort import realsorted as rst
from matplotlib.ticker import FormatStrFormatter
import ptitprince as pt
import pandas as pd
import seaborn as sns
import plotly.express as px

In [2]:
#Usefull functions
def contrastNoiseRatio(dataList, preStim, onStim):
    meanBL = np.mean(dataList[0:preStim], axis=0)
    meanSTIM = np.mean(dataList[preStim:(preStim+onStim)], axis=0)
    return (meanSTIM - meanBL)/np.std(dataList[0:preStim])

def linearFit(L, y):
    x = np.linspace(0, len(y), num = len(y))
    if L == '_l1':
        # find L1 line fit
        l1_fit = lambda x0, x, y: np.sum(np.abs(x0[0] * x + x0[1] - y))
        X1 = spopt.fmin(func=l1_fit, x0=[1, 1], args=(x, y))
        print(X1)
        return X1[0]*x + X1[1]
    elif L == '_l2':
        # find L2 line fit
        l2_fit = lambda x0, x, y: np.sum(np.power(x0[0] * x + x0[1] - y, 2))
        X2 = spopt.fmin(func=l2_fit, x0=[1, 1], args=(x, y))
        print(X2)
        return X2[0]*x + X2[1]
    else:
        print('No value for method provided')
        
def plotHalfViolin(dataToPlot, lgnd, groupedBy):
    
    plt.rcParams['figure.figsize'] = [30, 16]
    plt.rcParams.update({'font.size': 30})
    
    grp_by = groupedBy
    
    R = r'$\overline ' + '{' + f'{lgnd}' + '}' + '$ = '
    
    fig, ax = plt.subplots()

    ort="v"; pal = sns.color_palette("Set2", 4)
    
    if session == 'N_ROIs':
        if grp_by:
            plt.title(f'{lgnd} of {grp_by} trials average: ' + f' mouse-{subject},' + ' negative BOLD')
        else:
            plt.title(f'{lgnd}: ' + f' mouse-{subject},' + ' negative BOLD')
    else:
        if grp_by:
            plt.title(f'{lgnd} of {grp_by} trials average: ' + f' mouse-{subject},' + ' positive BOLD')
        else:
            plt.title(f'{lgnd}: ' + f' mouse-{subject},' + ' positive BOLD')
    
    plt.ylabel(lgnd)

    ax=pt.half_violinplot(data = dataToPlot, bw = .2, palette = pal, cut=True,
                          scale = "area", width = .6, inner = None, orient=ort)

    if len(ROIs) == 3:
        plt.legend ((R + f'{round(np.mean(dataToPlot)[0],2)}', 
                     R + f'{round(np.mean(dataToPlot)[1],2)}', 
                     R + f'{round(np.mean(dataToPlot)[2],2)}'),
                       loc='upper center', bbox_to_anchor=(0.5, 1.00),
                          ncol=3, fancybox=True)
    if len(ROIs) == 4:
        plt.legend ((R + f'{round(np.mean(dataToPlot)[0],2)}', 
                     R + f'{round(np.mean(dataToPlot)[1],2)}', 
                     R + f'{round(np.mean(dataToPlot)[2],2)}',
                     R + f'{round(np.mean(dataToPlot)[3],2)}'),  
                     loc='upper center', bbox_to_anchor=(0.5, 1.00),
                          ncol=4, fancybox=True)
    
    if np.min(np.min(dataToPlot))>1 :
        ax.set_ylim(np.min(np.min(dataToPlot))*.75, np.max(np.max(dataToPlot))*1.15)
    else:
        ax.set_ylim(np.min(np.min(dataToPlot))-0.5, np.max(np.max(dataToPlot))+1.0)

    ax=sns.stripplot(data = dataToPlot, palette = pal, edgecolor = "gray",
                     size = 20, jitter = 0.06, zorder = 0, orient=ort)
    ax=sns.boxplot(data = dataToPlot, color = "black", width = .15, zorder = 10,\
                showcaps = True, boxprops = {'facecolor':'none', "zorder":10},\
                showfliers=True, whiskerprops = {'linewidth':2, "zorder":10},\
                   saturation = 1, orient=ort)
    return fig, ax

def plotBar(dataToPlot, lgnd, groupedBy):
    
    plt.rcParams['figure.figsize'] = [30, 16]
    plt.rcParams.update({'font.size': 30})
        
    grp_by = groupedBy
    
    R = r'$\overline ' + '{' + f'{lgnd}' + '}' + '$ = '
    
    fig, ax = plt.subplots()
            
    pal = sns.color_palette("Set2", 4)
    
    if session == 'N_ROIs':
        if grp_by:
            plt.title(f'{lgnd} of {grp_by} trials average: ' + f' mouse-{subject},' + ' negative BOLD')
        else:
            plt.title(f'{lgnd}: ' + f' mouse-{subject},' + ' negative BOLD')
    else:
        if grp_by:
            plt.title(f'{lgnd} of {grp_by} trials average: ' + f' mouse-{subject},' + ' positive BOLD')
        else:
            plt.title(f'{lgnd}: ' + f' mouse-{subject},' + ' positive BOLD')
    
    plt.ylabel(lgnd)
    
    
    ax = sns.swarmplot(data=dataToPlot, palette=pal, edgecolor="black", size=20)
    
    if len(ROIs) == 3:
        plt.legend ((R + f'{round(np.mean(dataToPlot)[0],2)}', 
                     R + f'{round(np.mean(dataToPlot)[1],2)}', 
                     R + f'{round(np.mean(dataToPlot)[2],2)}'),
                       loc='upper center', bbox_to_anchor=(0.5, 1.00),
                          ncol=3, fancybox=True)
    if len(ROIs) == 4:
        plt.legend ((R + f'{round(np.mean(dataToPlot)[0],2)}', 
                     R + f'{round(np.mean(dataToPlot)[1],2)}', 
                     R + f'{round(np.mean(dataToPlot)[2],2)}',
                     R + f'{round(np.mean(dataToPlot)[3],2)}'),  
                     loc='upper center', bbox_to_anchor=(0.5, 1.00),
                          ncol=4, fancybox=True)
    
    if np.min(np.min(dataToPlot))>1 :
        ax.set_ylim(np.min(np.min(dataToPlot))*.75, np.max(np.max(dataToPlot))*1.15)
    else:
        ax.set_ylim(np.min(np.min(dataToPlot))-0.5, np.max(np.max(dataToPlot))+1.0)
        
    ax = sns.boxplot(data = dataToPlot, color = "black", width = .5, zorder = 10,\
                showcaps = True, boxprops = {'linewidth':1,'facecolor':'none', "zorder":10},\
                showfliers=True, whiskerprops = {'linewidth':1, "zorder":10})
    
    
    return fig, ax


In [3]:
#GLOBAL CONSTs
savefigs = True
preStim = 40
onStim = 20

In [4]:
base_dir = '/home/brainy/Documents/15tdata/Processing/ROIs'
os.chdir(base_dir)


In [5]:
#Get SUBJECT name
subj = rst(os.listdir(base_dir))
print('Subjects in the list {} '.format(subj))

Subjects in the list ['M1', 'M2', 'M3'] 


In [6]:
#Get SESSIONS name
ses = {}
for subject in subj:
    ses[subject] = rst(os.listdir(opj(base_dir, subject)))
    print(' Subject {} : {} '.format(subject, ses[subject]))

 Subject M1 : ['N_ROIs', 'P_ROIs', 'PbN_ROIs', 'Processed', 'QA_NEG', 'QA_POS'] 
 Subject M2 : ['N_ROIs', 'P_ROIs', 'PbN_ROIs', 'Processed', 'QA_NEG', 'QA_POS'] 
 Subject M3 : ['N_ROIs', 'P_ROIs', 'PbN_ROIs', 'Processed', 'QA_NEG', 'QA_POS'] 


In [7]:
#Get FILES in SUBJECT with specified SESSION 
subject = subj[1]
session = ses[subject][0]
work_dir = opj(base_dir, subject, session)
print(f'subj - {subject}, \nsession - {session}, \nfiles = {len(os.listdir(work_dir))}')

subj - M2, 
session - N_ROIs, 
files = 130


In [8]:
fileNames = [f for f in os.listdir(opj(base_dir, subject, session)) if f.endswith('.txt')]

ROIs = [str.split(name, '_')[0] for name in fileNames]
ROIs = rst(list(dict.fromkeys(ROIs)))

print(f'Regions of interest - {ROIs}')
#roi_slice = [str.split(name, '_')[1] for name in fileNames]

Regions of interest - ['barrFieldAud-L', 'barrelField-L', 'barrelField-R']


In [9]:
#For group averaging
grfilesDict = {}
grAverData = {}

for ROI in ROIs:
    #print(ROI)
    sink_1 = []
    for sub in subj:
        #print(sub)
        sink_1.append(rst(glob(opj(base_dir, sub, session + '/' + '*' + ROI + '*.txt'))[0:40]))
        #print(np.shape(sink))
    grfilesDict[ROI] = sink_1
    
    sink_2 = []
    for j in range(np.shape(sink_1)[1]):
        l = []
        [l.append(np.loadtxt(grfilesDict[ROI][i][j])) for i in range(len(subj))]
        sink_2.append(np.mean(l, axis=0))
        #print(np.shape(l))
    grAverData[ROI] = sink_2

In [10]:
#Group results by BLOCKS - 
if session == 'N_ROIs':
    group_by = 5
else:
    group_by = 3

blockDict = {}
for ROI in ROIs:
    blockDict[ROI] = list(zip(*[iter(grAverData[ROI])]*group_by))

In [11]:
# X for time Course
x_main = np.linspace(0, np.shape(blockDict[ROIs[0]])[2], 
                     num = np.shape(blockDict[ROIs[0]])[2])

np.shape(blockDict[ROI])

(8, 5, 120)